In [1]:
pip install python-chess

In [2]:
import chess.pgn

In [10]:
from stockfish import Stockfish
import chess
stockfish=Stockfish('C:/Users/Administrator/Desktop/stockfish-windows-x86-64-avx2/stockfish/stockfish-windows-x86-64-avx2.exe')

In [10]:
pgn_file='smaller.pgn'
game_count=0
with open (pgn_file) as file:
    while True:
        game=chess.pgn.read_game(file)
        if game is None:
            break
        game_count+=1
print(f"number of games in smaller.pgn:{game_count}")

number of games in smaller.pgn:3


In [22]:
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Current timestamp: {current_time}")

Current timestamp: 2025-02-12 15:32:16


In [23]:
#不要这个main()，然后循环求其他比赛
from stockfish import Stockfish
import chess
import chess.pgn

# 设置 Stockfish 引擎路径
stockfish = Stockfish('C:/Users/Administrator/Desktop/stockfish-windows-x86-64-avx2/stockfish/stockfish-windows-x86-64-avx2.exe')

# 直接执行的逻辑
stockfish.set_depth(20)

games_detail = []
pgn_file = 'smaller.pgn'  # 请确保此处文件路径正确
with open(pgn_file) as pgn:
    for i in range(3):  # 只读取前三局棋
        board = chess.Board()
        move_scores = []
        Moves = []
        
        game = chess.pgn.read_game(pgn)
        if game is None:  # 如果文件中没有更多的游戏，退出
            break
        
        for move in game.mainline_moves():
            board.push(move)
            stockfish.set_fen_position(board.fen())
            evaluation = stockfish.get_evaluation()
            
            if evaluation['type'] == 'cp':
                score = evaluation['value']
            elif evaluation['type'] == 'mate':
                score = evaluation['value']
            
            move_scores.append(score / 500)  # 假设你想对分数做归一化
            Moves.append(move.uci())
        
        out_dict = {}
        out_dict["Evaluation"] = move_scores
        out_dict["Black"] = game.headers['Black']
        out_dict['White'] = game.headers['White']
        out_dict['Date'] = game.headers['Date']
        out_dict['Competition'] = game.headers['Result']
        out_dict['Round'] = game.headers['Round']
        out_dict['Moves'] = Moves
        games_detail.append(out_dict)

# 打印最终结果
print(games_detail)
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Current timestamp: {current_time}")

[{'Evaluation': [0.058, 0.076, 0.066, 0.074, 0.074, 0.062, 0.058, 0.058, 0.038, 0.06, 0.076, 0.072, 0.062, 0.082, 0.0, 0.0, 0.0, 0.0, -0.046, -0.042, -0.052, -0.026, -0.038, -0.014, -0.034, -0.04, -0.178, -0.158, -0.188, -0.136, -0.132, -0.182, -0.302, -0.308, -0.354, -0.232, -0.222, -0.25, -0.288, -0.248, -0.314, -0.322, -0.332, -0.33, -0.25, -0.318, -0.37, -0.222, -0.224, -0.138, -0.208, -0.248, -0.216, -0.242, -0.206, -0.248, -0.216, -0.236, -0.228, -0.244, -0.238, -0.102, -0.364, 0.156, 0.112, 0.164, 0.048, 0.0, 0.126, 0.068, 0.048, 0.214, 0.036, 0.282, 0.28, 0.284, 0.314, 0.402, 0.362, 0.688, 0.452, 0.962, 0.41, 0.452, 0.272, 1.018, 1.12, 1.088, 0.844, 0.984, 0.948, 0.024, 0.678, 0.97, 0.74, 1.322, 0.018, 0.008, 0.006, 0.006, 0.004, 0.004, 0.002, 0.002, 0.0], 'Black': 'Barsov, Alexei', 'White': 'Fawzy, Adham', 'Date': '2025.01.07', 'Competition': '1-0', 'Round': '3', 'Moves': ['e2e4', 'e7e6', 'd2d4', 'd7d5', 'e4e5', 'c7c5', 'c2c3', 'b8c6', 'g1f3', 'd8b6', 'f1e2', 'c5d4', 'c3d4', '

# 循环求3 games大约耗费2mins20secs

In [51]:
from mpi4py import MPI
from stockfish import Stockfish
import chess
import chess.pgn

# MPI 初始化
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# 设置 Stockfish 引擎路径
STOCKFISH_PATH = 'C:/Users/Administrator/Desktop/stockfish-windows-x86-64-avx2/stockfish/stockfish-windows-x86-64-avx2.exe'

if rank == 0:
    # 主进程读取 PGN 文件
    pgn_file = 'smaller.pgn'
    num_games = 3  # 读取前三局
    games = []

    with open(pgn_file) as pgn:
        for _ in range(num_games):
            game = chess.pgn.read_game(pgn)
            if game is None:
                break
            games.append(game)

    # 计算每个进程分配多少个棋局
    chunk_size = (len(games) + size - 1) // size  # 向上取整
    chunks = [games[i * chunk_size:(i + 1) * chunk_size] for i in range(size)]

else:
    chunks = None

# 进程间广播棋局数据
local_games = comm.scatter(chunks, root=0)

# 每个进程创建自己的 Stockfish 实例
stockfish = Stockfish(STOCKFISH_PATH)
stockfish.set_depth(20)

local_results = []

for game in local_games:
    board = chess.Board()
    move_scores = []
    Moves = []

    for move in game.mainline_moves():
        board.push(move)
        stockfish.set_fen_position(board.fen())
        evaluation = stockfish.get_evaluation()

        if evaluation['type'] == 'cp':
            score = evaluation['value']
        elif evaluation['type'] == 'mate':
            score = evaluation['value']
        else:
            score = 0  # 其他情况归 0

        move_scores.append(score / 500)  # 归一化
        Moves.append(move.uci())

    out_dict = {
        "Evaluation": move_scores,
        "Black": game.headers.get('Black', 'Unknown'),
        "White": game.headers.get('White', 'Unknown'),
        "Date": game.headers.get('Date', 'Unknown'),
        "Competition": game.headers.get('Result', 'Unknown'),
        "Round": game.headers.get('Round', 'Unknown'),
        "Moves": Moves
    }

    local_results.append(out_dict)

# 收集所有进程的结果
all_results = comm.gather(local_results, root=0)

if rank == 0:
    # 展平结果
    games_detail = [game for sublist in all_results for game in sublist]
    print(games_detail)


[{'Evaluation': [0.058, 0.076, 0.066, 0.074, 0.074, 0.062, 0.058, 0.058, 0.038, 0.06, 0.076, 0.072, 0.062, 0.082, 0.0, 0.0, 0.0, 0.0, -0.046, -0.042, -0.052, -0.026, -0.038, -0.014, -0.034, -0.04, -0.178, -0.158, -0.188, -0.136, -0.132, -0.182, -0.302, -0.308, -0.354, -0.232, -0.222, -0.25, -0.288, -0.248, -0.314, -0.322, -0.332, -0.33, -0.25, -0.318, -0.37, -0.222, -0.224, -0.138, -0.208, -0.248, -0.216, -0.242, -0.206, -0.248, -0.216, -0.236, -0.228, -0.244, -0.238, -0.102, -0.364, 0.156, 0.112, 0.164, 0.048, 0.0, 0.126, 0.068, 0.048, 0.214, 0.036, 0.282, 0.28, 0.284, 0.314, 0.402, 0.362, 0.688, 0.452, 0.962, 0.41, 0.452, 0.272, 1.018, 1.12, 1.088, 0.844, 0.984, 0.948, 0.024, 0.678, 0.97, 0.74, 1.322, 0.018, 0.008, 0.006, 0.006, 0.004, 0.004, 0.002, 0.002, 0.0], 'Black': 'Barsov, Alexei', 'White': 'Fawzy, Adham', 'Date': '2025.01.07', 'Competition': '1-0', 'Round': '3', 'Moves': ['e2e4', 'e7e6', 'd2d4', 'd7d5', 'e4e5', 'c7c5', 'c2c3', 'b8c6', 'g1f3', 'd8b6', 'f1e2', 'c5d4', 'c3d4', '

In [ ]:
#用MPI加快运算
!pip install mpi4py
from mpi4py import MPI

In [46]:
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Current timestamp: {current_time}")

Current timestamp: 2025-02-13 09:37:03


In [47]:
import chess
import chess.pgn

In [48]:
comm=MPI.COMM_WORLD
rank=comm.Get_rank()
size=comm.Get_size()
if rank == 0:
    with open('smaller.pgn', 'r') as pgn_file:
        games = []
        while True:
            game = chess.pgn.read_game(pgn_file)
            if game is None:
                break
            games.append(game)
else:
    games = None

games=comm.bcast(games,root=0)

num_games=len(games)
games_per_process=num_games//size
start_idx=rank*games_per_process
end_idx = (rank + 1) * games_per_process if rank != size - 1 else num_games
local_games_detail = []  # 用于存储当前进程处理的棋局的评估结果
for i in range(start_idx, end_idx):
    game = games[i]
    board = chess.Board()  # 初始化一个空棋盘
    move_scores = []  # 用于存储每步棋的评分
    Moves = []  # 用于存储每步棋的 UCI 格式表示
    
    for move in game.mainline_moves():
        board.push(move)  # 推进棋局，执行一棋步
        stockfish.set_fen_position(board.fen())  # 将棋盘转换为 FEN 格式，并传给 Stockfish 引擎
        evaluation = stockfish.get_evaluation()  # 获取评估结果
        
        if evaluation['type'] == 'cp':
            score = evaluation['value']  # 获取评分值（cp：中心化评分）
        elif evaluation['type'] == 'mate':
            score = evaluation['value']  # 获取评分值（mate：局势胜负）
        
        move_scores.append(score / 500)  # 将评分归一化（假设评分范围较大，归一化有助于统一尺度）
        Moves.append(move.uci())  # 将棋步转换为 UCI 格式
    
    # 将当前棋局的评估信息整理为字典
    out_dict = {
        "Evaluation": move_scores,
        "Black": game.headers['Black'],
        "White": game.headers['White'],
        "Date": game.headers['Date'],
        "Competition": game.headers['Result'],
        "Round": game.headers['Round'],
        "Moves": Moves
    }
    
    local_games_detail.append(out_dict)  # 将当前棋局的评估信息添加到列表中

In [49]:
from datetime import datetime

# 获取当前日期和时间并格式化
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Current timestamp: {current_time}")

Current timestamp: 2025-02-13 09:39:26


In [50]:
local_games_detail

[{'Evaluation': [0.058,
   0.076,
   0.066,
   0.074,
   0.074,
   0.062,
   0.058,
   0.058,
   0.038,
   0.06,
   0.076,
   0.072,
   0.062,
   0.082,
   0.0,
   0.0,
   0.0,
   0.0,
   -0.046,
   -0.042,
   -0.052,
   -0.026,
   -0.038,
   -0.014,
   -0.034,
   -0.04,
   -0.178,
   -0.158,
   -0.188,
   -0.136,
   -0.132,
   -0.182,
   -0.302,
   -0.308,
   -0.354,
   -0.232,
   -0.222,
   -0.25,
   -0.288,
   -0.248,
   -0.314,
   -0.322,
   -0.332,
   -0.33,
   -0.25,
   -0.318,
   -0.37,
   -0.222,
   -0.224,
   -0.138,
   -0.208,
   -0.248,
   -0.216,
   -0.242,
   -0.206,
   -0.248,
   -0.216,
   -0.236,
   -0.228,
   -0.244,
   -0.238,
   -0.102,
   -0.364,
   0.156,
   0.112,
   0.164,
   0.048,
   0.0,
   0.126,
   0.068,
   0.048,
   0.214,
   0.036,
   0.282,
   0.28,
   0.284,
   0.314,
   0.402,
   0.362,
   0.688,
   0.452,
   0.962,
   0.41,
   0.452,
   0.272,
   1.018,
   1.12,
   1.088,
   0.844,
   0.984,
   0.948,
   0.024,
   0.678,
   0.97,
   0.74,
   1.322,
  

In [45]:
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Current timestamp: {current_time}")

Current timestamp: 2025-02-12 17:30:48


In [26]:
import chess
import chess.pgn
comm=MPI.COMM_WORLD
rank=comm.Get_rank()
size=comm.Get_size()
if rank == 0:
    with open('111.pgn', 'r') as pgn_file:
        games = []
        while True:
            game = chess.pgn.read_game(pgn_file)
            if game is None:
                break
            games.append(game)
else:
    games = None

games=comm.bcast(games,root=0)

num_games=len(games)
games_per_process=num_games//size
start_idx=rank*games_per_process
end_idx = (rank + 1) * games_per_process if rank != size - 1 else num_games
local_games_detail = []  # 用于存储当前进程处理的棋局的评估结果
for i in range(start_idx, end_idx):
    game = games[i]
    board = chess.Board()  # 初始化一个空棋盘
    move_scores = []  # 用于存储每步棋的评分
    Moves = []  # 用于存储每步棋的 UCI 格式表示
    
    for move in game.mainline_moves():
        board.push(move)  # 推进棋局，执行一棋步
        stockfish.set_fen_position(board.fen())  # 将棋盘转换为 FEN 格式，并传给 Stockfish 引擎
        evaluation = stockfish.get_evaluation()  # 获取评估结果
        
        if evaluation['type'] == 'cp':
            score = evaluation['value']  # 获取评分值（cp：中心化评分）
        elif evaluation['type'] == 'mate':
            score = evaluation['value']  # 获取评分值（mate：局势胜负）
        
        move_scores.append(score / 500)  # 将评分归一化（假设评分范围较大，归一化有助于统一尺度）
        Moves.append(move.uci())  # 将棋步转换为 UCI 格式
    
    # 将当前棋局的评估信息整理为字典
    out_dict = {
        "Evaluation": move_scores,
        "Black": game.headers['Black'],
        "White": game.headers['White'],
        "Date": game.headers['Date'],
        "Competition": game.headers['Result'],
        "Round": game.headers['Round'],
        "Moves": Moves
    }
    
    local_games_detail.append(out_dict)  # 将当前棋局的评估信息添加到列表中
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Current timestamp: {current_time}")

Current timestamp: 2025-02-12 16:27:54


# 用mpi加速一个含有12局game的111.pgn文件后，耗时约9mins

In [33]:
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Current timestamp: {current_time}")

Current timestamp: 2025-02-12 17:12:21


In [34]:
import chess
import chess.pgn
comm=MPI.COMM_WORLD
rank=comm.Get_rank()
size=comm.Get_size()
if rank == 0:
    with open('6_games.pgn', 'r') as pgn_file:
        games = []
        while True:
            game = chess.pgn.read_game(pgn_file)
            if game is None:
                break
            games.append(game)
else:
    games = None

games=comm.bcast(games,root=0)

num_games=len(games)
games_per_process=num_games//size
start_idx=rank*games_per_process
end_idx = (rank + 1) * games_per_process if rank != size - 1 else num_games
local_games_detail = []  # 用于存储当前进程处理的棋局的评估结果
for i in range(start_idx, end_idx):
    game = games[i]
    board = chess.Board()  # 初始化一个空棋盘
    move_scores = []  # 用于存储每步棋的评分
    Moves = []  # 用于存储每步棋的 UCI 格式表示
    
    for move in game.mainline_moves():
        board.push(move)  # 推进棋局，执行一棋步
        stockfish.set_fen_position(board.fen())  # 将棋盘转换为 FEN 格式，并传给 Stockfish 引擎
        evaluation = stockfish.get_evaluation()  # 获取评估结果
        
        if evaluation['type'] == 'cp':
            score = evaluation['value']  # 获取评分值（cp：中心化评分）
        elif evaluation['type'] == 'mate':
            score = evaluation['value']  # 获取评分值（mate：局势胜负）
        
        move_scores.append(score / 500)  # 将评分归一化（假设评分范围较大，归一化有助于统一尺度）
        Moves.append(move.uci())  # 将棋步转换为 UCI 格式
    
    # 将当前棋局的评估信息整理为字典
    out_dict = {
        "Evaluation": move_scores,
        "Black": game.headers['Black'],
        "White": game.headers['White'],
        "Date": game.headers['Date'],
        "Competition": game.headers['Result'],
        "Round": game.headers['Round'],
        "Moves": Moves
    }
    
    local_games_detail.append(out_dict)  # 将当前棋局的评估信息添加到列表中
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Current timestamp: {current_time}")

Current timestamp: 2025-02-12 17:16:59


# 用MPI加速一个含6个game的pgn文件后，耗时约4mins30secs

In [35]:
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Current timestamp: {current_time}")

Current timestamp: 2025-02-12 17:19:18


In [36]:
import chess
import chess.pgn
comm=MPI.COMM_WORLD
rank=comm.Get_rank()
size=comm.Get_size()
if rank == 0:
    with open('4_games.pgn', 'r') as pgn_file:
        games = []
        while True:
            game = chess.pgn.read_game(pgn_file)
            if game is None:
                break
            games.append(game)
else:
    games = None

games=comm.bcast(games,root=0)

num_games=len(games)
games_per_process=num_games//size
start_idx=rank*games_per_process
end_idx = (rank + 1) * games_per_process if rank != size - 1 else num_games
local_games_detail = []  # 用于存储当前进程处理的棋局的评估结果
for i in range(start_idx, end_idx):
    game = games[i]
    board = chess.Board()  # 初始化一个空棋盘
    move_scores = []  # 用于存储每步棋的评分
    Moves = []  # 用于存储每步棋的 UCI 格式表示
    
    for move in game.mainline_moves():
        board.push(move)  # 推进棋局，执行一棋步
        stockfish.set_fen_position(board.fen())  # 将棋盘转换为 FEN 格式，并传给 Stockfish 引擎
        evaluation = stockfish.get_evaluation()  # 获取评估结果
        
        if evaluation['type'] == 'cp':
            score = evaluation['value']  # 获取评分值（cp：中心化评分）
        elif evaluation['type'] == 'mate':
            score = evaluation['value']  # 获取评分值（mate：局势胜负）
        
        move_scores.append(score / 500)  # 将评分归一化（假设评分范围较大，归一化有助于统一尺度）
        Moves.append(move.uci())  # 将棋步转换为 UCI 格式
    
    # 将当前棋局的评估信息整理为字典
    out_dict = {
        "Evaluation": move_scores,
        "Black": game.headers['Black'],
        "White": game.headers['White'],
        "Date": game.headers['Date'],
        "Competition": game.headers['Result'],
        "Round": game.headers['Round'],
        "Moves": Moves
    }
    
    local_games_detail.append(out_dict)  # 将当前棋局的评估信息添加到列表中
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Current timestamp: {current_time}")

Current timestamp: 2025-02-12 17:22:30


# 用MPI加速一个4_games.pgn文件后，耗时约3mins10secs

In [37]:
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Current timestamp: {current_time}")

Current timestamp: 2025-02-12 17:23:56


In [38]:
import chess
import chess.pgn
comm=MPI.COMM_WORLD
rank=comm.Get_rank()
size=comm.Get_size()
if rank == 0:
    with open('3_games.pgn', 'r') as pgn_file:
        games = []
        while True:
            game = chess.pgn.read_game(pgn_file)
            if game is None:
                break
            games.append(game)
else:
    games = None

games=comm.bcast(games,root=0)

num_games=len(games)
games_per_process=num_games//size
start_idx=rank*games_per_process
end_idx = (rank + 1) * games_per_process if rank != size - 1 else num_games
local_games_detail = []  # 用于存储当前进程处理的棋局的评估结果
for i in range(start_idx, end_idx):
    game = games[i]
    board = chess.Board()  # 初始化一个空棋盘
    move_scores = []  # 用于存储每步棋的评分
    Moves = []  # 用于存储每步棋的 UCI 格式表示
    
    for move in game.mainline_moves():
        board.push(move)  # 推进棋局，执行一棋步
        stockfish.set_fen_position(board.fen())  # 将棋盘转换为 FEN 格式，并传给 Stockfish 引擎
        evaluation = stockfish.get_evaluation()  # 获取评估结果
        
        if evaluation['type'] == 'cp':
            score = evaluation['value']  # 获取评分值（cp：中心化评分）
        elif evaluation['type'] == 'mate':
            score = evaluation['value']  # 获取评分值（mate：局势胜负）
        
        move_scores.append(score / 500)  # 将评分归一化（假设评分范围较大，归一化有助于统一尺度）
        Moves.append(move.uci())  # 将棋步转换为 UCI 格式
    
    # 将当前棋局的评估信息整理为字典
    out_dict = {
        "Evaluation": move_scores,
        "Black": game.headers['Black'],
        "White": game.headers['White'],
        "Date": game.headers['Date'],
        "Competition": game.headers['Result'],
        "Round": game.headers['Round'],
        "Moves": Moves
    }
    
    local_games_detail.append(out_dict)  # 将当前棋局的评估信息添加到列表中
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Current timestamp: {current_time}")

Current timestamp: 2025-02-12 17:26:37


# 用MPI加速一个3_games.pgn后，耗时约2mins20secs